In [5]:
#BALLARD SPAHR
import requests
from bs4 import BeautifulSoup
baseurl= "https://www.ballardspahr.com/People"
url="https://www.ballardspahr.com/People?Services=%7BB160CF7E-CBE1-4342-85FF-FF2A86B7A235%7D"
headers ={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
    }
#request
res = requests.get(url=baseurl, headers=headers)
soup = BeautifulSoup(res.content, 'lxml')
#creating empty list
servicedata=[]
namedata=[]
officedata=[]
roledata=[]
emaildata=[]
telenumberdata=[]
faxnumberdata=[]
pagelink=[]
services=[]
#services
for option in soup.find('select', {'title':'Filter by Services'}).find_all('option'):
    if option.get('value'):
        services.append((option.get('value'), option.text))
for service_id, service_name in services[1:6]:
        service_url =baseurl+f"?Services={service_id}"
        res = requests.get(url=service_url, headers=headers)
        soup = BeautifulSoup(res.content, 'lxml')
        for option in soup.find('select',{'title':'Filter by Offices'}).find_all('option')[1:]:
            if option.get('value'):
                office_id = option.get('value')
                for page_num in range(5):
                    full_url = f"https://www.ballardspahr.com/sitecore/api/people/search?lang=en&sc_apikey=%7B8BEE2997-A9B1-4874-A4C3-7EBA04C493EC%7D&page={page_num}&Services={service_id}&Offices={office_id}"
                    res = requests.get(full_url, headers=headers)
                    if res.ok:
                        for i in res.json()['Results']:
                            person_url = i['url']
                            final_url = f"https://www.ballardspahr.com/sitecore/api/layout/render/jss?lang=en&sc_apikey=%7B8BEE2997-A9B1-4874-A4C3-7EBA04C493EC%7D&item={person_url}"
                            res = requests.get(url=final_url, headers=headers)
                            json_object = res.json()['sitecore']['route']
                            servicesname=service_name
                            servicedata.append(servicesname)
                            name=json_object['placeholders']['head'][0]['fields']['Title']
                            namedata.append(name.rsplit('|', 2)[0]) 
                            role=json_object['placeholders']['content'][0]['placeholders']['aside'][0]['fields']['Title']['fields']['Name']['value']
                            roledata.append(role.strip('()'))
                            email=json_object['placeholders']['content'][0]['placeholders']['aside'][0]['fields']['Email']['value']               
                            emaildata.append(email.strip('()'))
                            office=option.text
                            officedata.append(office)
                            telenumber=json_object['placeholders']['content'][0]['placeholders']['aside'][1]['placeholders']['related-offices'][0]['fields']['OfficeNumber']['value']
                            telenumberdata.append(telenumber.strip('()'))
                            faxnumber=json_object['placeholders']['content'][0]['placeholders']['aside'][1]['placeholders']['related-offices'][0]['fields']['FaxNumber']['value']
                            faxnumberdata.append(faxnumber.strip('()'))
                            pageurl=baseurl.strip('/People')+person_url
                            pagelink.append(pageurl)
                            #END

In [24]:
#DATAFRAME
import pandas as pd
from datetime import datetime

maindata = pd.DataFrame()
maindata['SNO'] = maindata.index + 1
maindata.set_index('SNO',inplace=True)
maindata['SERVICES'] =servicedata
maindata['NAME'] =namedata
maindata['ROLE'] =roledata
maindata['EMAIL'] = emaildata
maindata['OFFICES'] =officedata
maindata['TELEPHONE_NUMBER'] =telenumberdata
maindata['FAX_NUMBER'] =faxnumberdata
maindata['PAGE_URL'] =pagelink
maindata['DATE_TIME']= pd.Timestamp(datetime.now())
maindata


,SERVICES,NAME,ROLE,EMAIL,OFFICES,TELEPHONE_NUMBER,FAX_NUMBER,PAGE_URL,DATE_TIME
SNO,,,,,,,,,
0,Accessibility,Timothy F. McCormack,Senior Counsel,mccormackt@ballardspahr.com,Baltimore,410.528.5680,410.528.5650,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
1,Accessibility,Michelle M. McGeogh,Partner,mcgeoghm@ballardspahr.com,Baltimore,410.528.5661,410.528.5650,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
2,Accessibility,Maraya N. Pratt,Associate,prattmn@ballardspahr.com,Baltimore,410.528.5578,410.528.5650,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
3,Accessibility,Jessica G. Federico,Associate,federicoj@ballardspahr.com,Denver,303.299.7329,303.296.3956,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
4,Accessibility,Brian D. Huben,Partner,hubenb@ballardspahr.com,Los Angeles,424.204.4353,424.204.4350,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
5,Accessibility,Jenna K. Johnson,Associate,johnsonjk@ballardspahr.com,Minneapolis,612.371.3232,612.371.3207,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
6,Accessibility,Mary M. O'Brien,Of Counsel,obrienm@ballardspahr.com,Minneapolis,612.371.2401,612.371.3207,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
7,Accessibility,Lori Sommerfield,Of Counsel,sommerfieldl@ballardspahr.com,Minneapolis,612.371.2450,612.371.3207,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253
8,Accessibility,Tara L. Humma,Associate,hummat@ballardspahr.com,New Jersey,856.761.3408,856.761.1020,https://www.ballardspahr.com/People/Attorneys/...,2021-11-22 22:15:47.638253


In [15]:
pd.set_option('display.max_rows',None)

In [17]:
import os
import sqlite3
from flask import jsonify
from flask_api import FlaskAPI
from flask_restful import Api
from sqlalchemy import create_engine
from flask import request
import json
#FLASK API
app=FlaskAPI(__name__)
api=Api(app)
connection=create_engine('sqlite:///ballardspahrdatabase.db')

#HOME ROUTE
@app.route('/', methods=['GET'])
def home():
    return json.dumps({'WEBSITE': 'ballardspahr'})
#SELECT

@app.route('/ballardspahrselect', methods=['GET'])
def select():
    connect= connection.connect()
    select= connect.execute("select * from LAWYERTABLE")
    return json.dumps([dict(db) for db in select])
#INSERT

@app.route('/ballardspahrinsert', methods=['GET'])
def insert():
    connect = connection.connect()
    maindata.to_sql('LAWYERTABLE',connect,if_exists='replace')
    message = {"Message " : "DATA INSERTED SUCCESSFULLY"}
    return message

#CREATE
@app.route('/ballardspahrcreate', methods=['GET'])
def create():
    try:
        connect = connection.connect()
        trans = connect.begin()
        connect.execute('CREATE TABLE IF NOT EXISTS LAWYERTABLE(SNO INT PRIMARY KEY ,SERVICES TEXT,NAME TEXT,ROLE TEXT,EMAIL TEXT,OFFICES TEXT,TELEPHONE_NUMBER INT,FAX_NUMBER INT,PAGE_URL TEXT,TIMESTAMP DATETIME DEFAULT CURRENT_TIMESTAMP )')
        trans.commit()
        message = {"Message " : "TABLE CREATED SUCCESSFULLY"}
        return message
    except Exception as error:
        message = {"Message " : "TABLE CREATION FAILED"}
        return message
    finally:
        connect.close()
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2021 21:26:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 21:27:24] "GET /ballardspahrcreate HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/css/default.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/js/jquery-1.8.1-min.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/css/prettify.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/js/prettify-min.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/css/bootstrap-tweaks.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/js/default.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "GET /flask-api/static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Nov/2021 21:27:26] "